In [95]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn import utils
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

from sklearn.model_selection import train_test_split

In [96]:
# path for the csv files
DATA_PATH = os.path.join(os.getcwd(), 'data')

# loading data to pandas dataframe
def load_data(file_name): 
    file_path = os.path.join(DATA_PATH, file_name) 
    return pd.read_csv(file_path, parse_dates = ['purchase_date', 'release_date'])

In [97]:
def extract_dateinfo(df, col_name):
    df[col_name+'_year'] = df.loc[:,col_name].apply(lambda x: x.year)
    df[col_name+'_month'] = df.loc[:,col_name].apply(lambda x: x.month)
    df[col_name+'_day'] = df.loc[:,col_name].apply(lambda x: x.day)
    
    return df

In [98]:
#Outlier detection
def detect_outliers(df, n, features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    
    outlier_indices = []
    
    for col in features:
        # calculating interquartile range
        Q1 = np.nanpercentile(df[col], 25)
        Q3 = np.nanpercentile(df[col], 75)
        IQR = Q3 - Q1
        print(IQR)
        
        outlier_step = 1.5 * IQR
        
        
        
        # get the indices of outliers for feature col
        outliers_in_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        
        # append the indices to oulier_indices
        outlier_indices.extend(outliers_in_col)
    
    outlier_indices = Counter(outlier_indices)
    result = list(k for k, v in outlier_indices.items() if v > n)
    
    return result

In [99]:
train_set = load_data('train.csv')
test_set = load_data('test.csv')




In [100]:
outliers_to_drop = detect_outliers(train_set, 2 ,['price', 'total_positive_reviews', 'total_negative_reviews'])
train_set.loc[outliers_to_drop]
train_set = train_set.drop(outliers_to_drop, axis = 0).reset_index(drop=True)

train_len = train_set.shape[0]
test_len = test_set.shape[0]

5400.0
8867.5
1105.0


In [101]:
# game_info.head()
game_info =  pd.concat(objs=[train_set, test_set], axis=0, sort=False).reset_index(drop=True)
game_info

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,0.000000,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.0,96.0
1,1,0.016667,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.0,0.0
2,2,0.000000,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.0,663.0
3,3,1.533333,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.0,1746.0
4,4,22.333333,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.0,523.0
...,...,...,...,...,...,...,...,...,...,...,...
436,85,NaN,False,5000.0,Action,"Single-player,Steam Achievements,Full controll...","Western,FPS,Action,Story Rich,Bullet Time,Shoo...",2018-03-23,2013-05-22,9334.0,516.0
437,86,NaN,False,3600.0,"Action,Adventure",Single-player,"Adventure,Action,Ninja,Stealth,Cute,Singleplay...",2018-03-03,2009-09-14,1245.0,176.0
438,87,NaN,False,11200.0,"Simulation,Strategy","Single-player,Multi-player,Cross-Platform Mult...","Grand Strategy,Strategy,Historical,Simulation,...",2018-02-18,2013-08-13,33128.0,6350.0
439,88,NaN,True,0.0,"Action,Adventure,RPG,Simulation,Sports,Strategy","Single-player,Multi-player,Co-op,Online Co-op,...","VR,RPG,Action,Sports,Simulation,Adventure,Stra...",2017-11-19,2017-06-19,5.0,0.0


In [102]:
# check null values
game_info.fillna(np.nan, inplace=True)

#fill missing purchase date with the most frequent value in purchase_date column
game_info['purchase_date'].fillna(game_info['purchase_date'].mode()[0], inplace=True)

#fille missing number of positive_reviews and negative_reviews with zeros
game_info['total_positive_reviews'].fillna(0.0, inplace=True)
game_info['total_negative_reviews'].fillna(0.0, inplace=True)

#transfer boolean values to 1(true) and 0(false)
game_info['is_free'] = game_info['is_free'].map({False: 0.0, True: 1.0})

#drop outliers


# split strings in the categorical columns
game_info['genres'] = game_info['genres'].str.split(',')
game_info['categories'] = game_info['categories'].str.split(',')
game_info['tags'] = game_info['tags'].str.split(',')

game_info

#game_info.isnull().sum()

,id,playtime_forever,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
0,0,0.000000,0.0,3700.0,"[Adventure, Casual, Indie]","[Single-player, Steam Trading Cards, Steam Cloud]","[Indie, Adventure, Story Rich, Casual, Atmosph...",2018-07-02,2013-12-10,372.0,96.0
1,1,0.016667,1.0,0.0,[RPG],"[Single-player, Partial Controller Support]","[Mod, Utilities, RPG, Game Development, Single...",2016-11-26,2015-08-12,23.0,0.0
2,2,0.000000,0.0,5000.0,"[Adventure, Casual, Indie]","[Single-player, Full controller support, Steam...","[Point & Click, Adventure, Story Rich, Comedy,...",2018-07-02,2014-01-28,3018.0,663.0
3,3,1.533333,0.0,9900.0,"[Action, RPG]","[Single-player, Multi-player, Steam Achievemen...","[Medieval, RPG, Open World, Strategy, Sandbox,...",2016-11-28,2010-03-31,63078.0,1746.0
4,4,22.333333,0.0,4800.0,"[Action, Indie, Strategy]","[Single-player, Co-op, Steam Achievements, Ful...","[Tower Defense, Co-op, Action, Strategy, Onlin...",2018-03-04,2012-07-30,8841.0,523.0
...,...,...,...,...,...,...,...,...,...,...,...
436,85,NaN,0.0,5000.0,[Action],"[Single-player, Steam Achievements, Full contr...","[Western, FPS, Action, Story Rich, Bullet Time...",2018-03-23,2013-05-22,9334.0,516.0
437,86,NaN,0.0,3600.0,"[Action, Adventure]",[Single-player],"[Adventure, Action, Ninja, Stealth, Cute, Sing...",2018-03-03,2009-09-14,1245.0,176.0
438,87,NaN,0.0,11200.0,"[Simulation, Strategy]","[Single-player, Multi-player, Cross-Platform M...","[Grand Strategy, Strategy, Historical, Simulat...",2018-02-18,2013-08-13,33128.0,6350.0
439,88,NaN,1.0,0.0,"[Action, Adventure, RPG, Simulation, Sports, S...","[Single-player, Multi-player, Co-op, Online Co...","[VR, RPG, Action, Sports, Simulation, Adventur...",2017-11-19,2017-06-19,5.0,0.0


In [103]:
#dataframe with only categorical values
cate_df = game_info[['genres', 'categories', 'tags']]


# one_hot encoding categorical columns 
genres_df = pd.get_dummies(cate_df['genres'].apply(pd.Series).stack()).sum(level=0)
categories_df = pd.get_dummies(cate_df['categories'].apply(pd.Series).stack()).sum(level=0)
tags_df = pd.get_dummies(cate_df['tags'].apply(pd.Series).stack()).sum(level=0)

# contatenate categorical dataframes with game_info
game_df = pd.concat([game_info.drop(columns=['genres', 'categories', 'tags']), genres_df, categories_df, tags_df], axis=1, sort=False)
game_df

,id,playtime_forever,is_free,price,purchase_date,release_date,total_positive_reviews,total_negative_reviews,Action,Adventure,...,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0,0.000000,0.0,3700.0,2018-07-02,2013-12-10,372.0,96.0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,0.016667,1.0,0.0,2016-11-26,2015-08-12,23.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.000000,0.0,5000.0,2018-07-02,2014-01-28,3018.0,663.0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1.533333,0.0,9900.0,2016-11-28,2010-03-31,63078.0,1746.0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,22.333333,0.0,4800.0,2018-03-04,2012-07-30,8841.0,523.0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,85,NaN,0.0,5000.0,2018-03-23,2013-05-22,9334.0,516.0,1,0,...,0,0,0,0,0,1,0,0,0,0
437,86,NaN,0.0,3600.0,2018-03-03,2009-09-14,1245.0,176.0,1,1,...,0,0,0,0,0,0,0,0,0,0
438,87,NaN,0.0,11200.0,2018-02-18,2013-08-13,33128.0,6350.0,0,0,...,0,0,0,0,0,0,0,0,0,0
439,88,NaN,1.0,0.0,2017-11-19,2017-06-19,5.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [104]:
# drop duplicate columns
game_df = game_df.loc[:, ~game_df.columns.duplicated()]
game_df

,id,playtime_forever,is_free,price,purchase_date,release_date,total_positive_reviews,total_negative_reviews,Action,Adventure,...,Voxel,Walking Simulator,War,Wargame,Warhammer 40K,Western,World War I,World War II,Zombies,eSports
0,0,0.000000,0.0,3700.0,2018-07-02,2013-12-10,372.0,96.0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,0.016667,1.0,0.0,2016-11-26,2015-08-12,23.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0.000000,0.0,5000.0,2018-07-02,2014-01-28,3018.0,663.0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1.533333,0.0,9900.0,2016-11-28,2010-03-31,63078.0,1746.0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,22.333333,0.0,4800.0,2018-03-04,2012-07-30,8841.0,523.0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,85,NaN,0.0,5000.0,2018-03-23,2013-05-22,9334.0,516.0,1,0,...,0,0,0,0,0,1,0,0,0,0
437,86,NaN,0.0,3600.0,2018-03-03,2009-09-14,1245.0,176.0,1,1,...,0,0,0,0,0,0,0,0,0,0
438,87,NaN,0.0,11200.0,2018-02-18,2013-08-13,33128.0,6350.0,0,0,...,0,0,0,0,0,0,0,0,0,0
439,88,NaN,1.0,0.0,2017-11-19,2017-06-19,5.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [105]:
game_df = extract_dateinfo(game_df, 'purchase_date')
game_df = extract_dateinfo(game_df, 'release_date')
game_df.drop(columns=['purchase_date', 'release_date'], inplace=True)
#game_df.drop(['purchase_date'])
#'purchase_date' in game_df
#game_df.loc[:,'purchase_date']
game_df

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,id,playtime_forever,is_free,price,total_positive_reviews,total_negative_reviews,Action,Adventure,Animation & Modeling,Audio Production,...,World War I,World War II,Zombies,eSports,purchase_date_year,purchase_date_month,purchase_date_day,release_date_year,release_date_month,release_date_day
0,0,0.000000,0.0,3700.0,372.0,96.0,0,1,0,0,...,0,0,0,0,2018,7,2,2013,12,10
1,1,0.016667,1.0,0.0,23.0,0.0,0,0,0,0,...,0,0,0,0,2016,11,26,2015,8,12
2,2,0.000000,0.0,5000.0,3018.0,663.0,0,1,0,0,...,0,0,0,0,2018,7,2,2014,1,28
3,3,1.533333,0.0,9900.0,63078.0,1746.0,1,0,0,0,...,0,0,0,0,2016,11,28,2010,3,31
4,4,22.333333,0.0,4800.0,8841.0,523.0,1,0,0,0,...,0,0,0,0,2018,3,4,2012,7,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,85,NaN,0.0,5000.0,9334.0,516.0,1,0,0,0,...,0,0,0,0,2018,3,23,2013,5,22
437,86,NaN,0.0,3600.0,1245.0,176.0,1,1,0,0,...,0,0,0,0,2018,3,3,2009,9,14
438,87,NaN,0.0,11200.0,33128.0,6350.0,0,0,0,0,...,0,0,0,0,2018,2,18,2013,8,13
439,88,NaN,1.0,0.0,5.0,0.0,1,1,0,0,...,0,0,0,0,2017,11,19,2017,6,19


In [108]:
train_data = game_df[:train_len]
test_data = game_df[train_len:]

train_label = train_data['playtime_forever']
train_data = train_data.drop(columns=['playtime_forever'])

# split game_info into training and validating datasets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


test_data.drop(columns=['playtime_forever'],inplace=True)

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape : ', y_train.shape)
print('y_val shape : ', y_val.shape)

X_train shape:  (280, 356)
X_val shape:  (71, 356)
y_train shape :  (280,)
y_val shape :  (71,)


In [110]:
#lab_enc = preprocessing.LabelEncoder()
#train_encoded = lab_enc.fit_transform(y_train)
#val_encoded = lab_enc.fit_transform(y_val)

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_val)

y_test_pred = linreg.predict(test_data)

print("Mean squared error: %.2f"
      % mean_squared_error(y_val, y_pred))

print(y_test_pred)

Mean squared error: 654.97
[ -6.04891426  10.50493159 -39.37029071 -10.48661057   3.23690408
  34.41718337  22.01602616   0.67946802  19.76603138   8.83723885
   6.05199318  -2.32410735   5.06711956 -50.77403603  -1.37760857
  21.18455031  -2.77568198  -4.20643779  23.87214725 -26.71681378
  -8.3038336   22.23204223   7.25708553 -12.41526474  -1.29435318
 -69.73243236  18.90254667 -28.85647055   0.40611706 -21.65671452
  14.41236465  -0.36047567 -32.52676216   6.98695512 -11.68874498
  16.51294023 -16.30516737  26.13060355  -1.07235096  11.9295633
  22.44262571   1.20336982  -4.1533696   14.05693383 -39.47184674
   9.02190526  23.04255771 -11.89088023 -12.98059335 -18.98334322
  19.33792925  -9.13953081  11.27595371 -14.67943832  22.60470421
 -25.22134632 -28.66496823   1.63235343  -5.8300388   -1.41562779
  41.39643643   6.67107091  -8.59893958  -5.49785785   6.15871558
  -5.03896041  -8.40188948  -1.94801832  11.46247839   3.13923126
   8.28438483  46.69811734  10.24565707  26.669723

In [119]:
import csv

result_df = pd.DataFrame(y_test_pred, columns =['playtime_forever']) 
result_df.index.name = 'id'
result_df.to_csv('result.csv')